In [1]:
import logging
logging.getLogger('tensorflow').disabled = True
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
!pip install bert-tensorflow

    100% |████████████████████████████████| 71kB 1.2MB/s ta 0:00:011


In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [4]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle

sys.path.append('../src')

from commons import elastic
from langdetect import detect
from text_processing import text_normalizer
from importlib import reload
from text_processing import duplicate_finder
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import Levenshtein
text_normalizer = reload(text_normalizer)
from dateutil.parser import parse
from utilities import excel_writer, excel_reader
from sklearn.model_selection import train_test_split

def get_train_and_test_data_splitted(temp_df, all_dataset = False, test_size=0.05):
    if all_dataset:
        train_data = temp_df
        test_data = temp_df
    else:
        train_data_x, test_data_x,train_y,test_y = train_test_split([data[:-1] for data in temp_df.values], [data[-1] for data in temp_df.values], test_size=test_size)
        train_data = np.concatenate((train_data_x,[[val]for val in train_y]),axis=1)
        test_data = np.concatenate((test_data_x,[[val] for val in test_y]),axis=1)
    return pd.DataFrame(train_data, columns=temp_df.columns), pd.DataFrame(test_data, columns=temp_df.columns)

In [5]:
!pip install scikit-multilearn==0.2.0

    100% |████████████████████████████████| 92kB 1.3MB/s ta 0:00:011


# Data

In [4]:
all_data = []
y = []
outcomes_data_folder = "../tmp/outcomes_data"
for file in os.listdir(outcomes_data_folder):
    df_outcome_mary = excel_reader.ExcelReader().read_df_from_excel(os.path.join(outcomes_data_folder, file))
    for i in range(len(df_outcome_mary)):
        outcomes = []
        outcomes_per_label = []
        outcomes_labels = []
        for column in ['Soil health', 'Fertilizer use', 'Water use', 'Gender',
           'Greenhouse gas emissions', 'Livelihood', 'Market access', 'Nutrition',
           'Production', 'Resilience', 'Knowledge sharing', 'Practice change',
           'Social inclusion', 'Poverty reduction', 'Environment impact']:
            outcomes_found = set([outcome for outcome in df_outcome_mary[column].values[i].split(";") if outcome.strip()])
            outcomes.extend(outcomes_found)
            for outcome in outcomes_found:
                outcomes_per_label.append((outcome, column))
            
            outcomes_labels.append(1 if len(outcomes_found) else 0)
        y.append(outcomes_labels)
        outcomes_found = set()
        for sent in nltk.sent_tokenize(df_outcome_mary["Sentence"].values[i]):
            found_entities = []
            found_outcome_plus_labels = []
            for outcome, label in outcomes_per_label:
                outcome = outcome.strip().strip(",").strip(".").strip()
                if outcome.lower() in sent.lower():
                    start_ind = sent.lower().rfind(outcome.lower())
                    end_ind = start_ind + len(outcome)
                    if sent[start_ind: end_ind].lower() != outcome.lower():
                        print(sent)
                        print(start_ind, end_ind)
                        print(sent[start_ind: end_ind])
                        print(sent[start_ind: end_ind].lower(), outcome.lower())
                    found_entities.append((start_ind, end_ind, 'OUTCOME'))
                    outcomes_found.add(outcome)
                    found_outcome_plus_labels.append((outcome, label))
            all_data.append((sent, {'entities': found_entities}, found_outcome_plus_labels))
        if len(outcomes_found) != len(set(outcomes)):
            print(outcomes, outcomes_found)
            print(i)
print("All data ", len(all_data))

All data  2622


In [5]:
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
X_train, y_train, X_test, y_test = iterative_train_test_split(np.asarray(all_data), np.asarray(y), test_size = 0.1)

In [6]:
def split_outcomes_data(merged_data):
    train_data = []
    train_data_outcomes = []
    train_sentences = []
    for data in merged_data:
        train_data.append((data[0], data[1]))
        train_data_outcomes.extend(data[2])
        train_sentences.append((data[0], [d[1] for d in data[2]]))
    return train_data, train_data_outcomes, train_sentences
        

TRAIN_DATA, outcomes_train_label_data, outcomes_sentences_train_data = split_outcomes_data(X_train)
EVAL_DATA, outcomes_eval_label_data, outcomes_sentences_eval_data = split_outcomes_data(X_test)
print("Train data", len(TRAIN_DATA))
print("Test data", len(EVAL_DATA))

Train data 2359
Test data 263


In [11]:
import pickle
from sklearn.utils import shuffle
TRAIN_DATA, outcomes_train_label_data, outcomes_sentences_train_data, EVAL_DATA, outcomes_eval_label_data, outcomes_sentences_eval_data = pickle.load(open("/hdd2/data/maryia_pavlovets/entity-recognition-model-more-2610_train_data", "rb"))
TRAIN_DATA = shuffle(TRAIN_DATA)
outcomes_train_label_data = shuffle(outcomes_train_label_data)
outcomes_sentences_train_data = shuffle(outcomes_sentences_train_data)

In [12]:
pd.DataFrame(outcomes_train_label_data, columns=["Text", "Label"])["Label"].value_counts()

Livelihood                  463
Production                  405
Greenhouse gas emissions    232
Market access               222
Nutrition                   208
Resilience                  198
Soil health                 179
Practice change             157
Gender                      147
Water use                   128
Fertilizer use              128
Knowledge sharing           125
Environment impact          117
Social inclusion            106
Poverty reduction           103
Name: Label, dtype: int64

In [7]:
pd.DataFrame(outcomes_eval_label_data, columns=["Text", "Label"])["Label"].value_counts()

Livelihood                  45
Production                  44
Nutrition                   26
Greenhouse gas emissions    24
Resilience                  22
Market access               21
Soil health                 21
Practice change             18
Gender                      16
Fertilizer use              14
Knowledge sharing           14
Environment impact          13
Social inclusion            13
Water use                   13
Poverty reduction           12
Name: Label, dtype: int64

In [4]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    #remove % and cast to float
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        print(data.max())
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [4]:
from outcomes_modelling import outcomes_full_logic_labeller
outcomes_full_logic_labeller = reload(outcomes_full_logic_labeller)
_outcomes_full_logic_labeller = outcomes_full_logic_labeller.OutcomesFullLogicLabeller(
    "/hdd2/data/maryia_pavlovets/bert_exp_outcome_new",
    "/hdd2/data/maryia_pavlovets/entity-recognition-model-more-2610")
_outcomes_full_logic_labeller.train_model("/hdd2/data/maryia_pavlovets/entity-recognition-model-more-2610_train_data")

Train data distribution
Livelihood                  463
Production                  405
Greenhouse gas emissions    232
Market access               222
Nutrition                   208
Resilience                  198
Soil health                 179
Practice change             157
Gender                      147
Water use                   128
Fertilizer use              128
Knowledge sharing           125
Environment impact          117
Social inclusion            106
Poverty reduction           103
Name: Label, dtype: int64
Valid data distribution
Livelihood                  45
Production                  44
Nutrition                   26
Greenhouse gas emissions    24
Resilience                  22
Market access               21
Soil health                 21
Practice change             18
Gender                      16
Knowledge sharing           14
Fertilizer use              14
Water use                   13
Social inclusion            13
Environment impact          13
Poverty redu

KeyboardInterrupt: 

In [7]:
_outcomes_full_logic_labeller.print_statistics()

Mean the best F1 score:  0.8390244371736914


,BERT(trained on only sentences with outcomes),BERT(trained on only sentences with outcomes) + proportion 0.1,BERT(trained on all sentences),BERT(trained on all sentences) + proportion 0.1,NER + BERT classification,NER + BERT classification + proportion 0.1,NER + BERT classification + keywords,NER + BERT classification + keywords + proportion 0.1
Label,,,,,,,,
Livelihood,0.857811,0.798102,0.842844,0.848658,0.794474,0.793373,0.461111,0.78344
Soil health,0.781344,0.764108,0.770126,0.812599,0.774884,0.812599,0.758439,0.812403
Fertilizer use,0.739247,0.708709,0.764108,0.800399,0.763799,0.770126,0.777693,0.777693
Water use,0.753986,0.702386,0.488576,0.655235,0.7987,0.811921,0.782706,0.7987
Gender,0.872981,0.872981,0.905519,0.87922,0.809573,0.809573,0.809573,0.809573
Greenhouse gas emissions,0.963873,0.963873,0.963873,0.950828,0.898983,0.887384,0.887384,0.898983
Market access,0.826407,0.766014,0.772003,0.763126,0.718841,0.744453,0.718841,0.744453
Nutrition,0.811667,0.812085,0.831076,0.83893,0.812085,0.79388,0.781344,0.82037
Production,0.74625,0.767275,0.827877,0.820729,0.766196,0.777837,0.76159,0.783514


In [8]:
_outcomes_full_logic_labeller.save_model()

In [13]:
def predict_proba(doc):
    return np.asarray(_outcomes_sentence_labeler.predict_for_df(pd.DataFrame([(d, -1) for d in doc], columns=["text", "label"]))[0])

In [98]:
idx = 149
text = outcomes_sentences_eval_data[idx][0]
print(text)
print(outcomes_sentences_eval_data[idx][1])

LCA results indicate that it is possible to reduce the GWP associated with refrigerated transportation by increasing local fish production and decreasing importation, especially given the available water potential of Bahia.
['Greenhouse gas emissions', 'Market access', 'Production']


In [99]:
from outcomes_modelling import outcomes_sentence_labeler
import eli5
from eli5.lime import TextExplainer

label = "Market access"
_outcomes_sentence_labeler = outcomes_sentence_labeler.OutcomesSentenceLabeler(
        "/hdd2/data/maryia_pavlovets/bert_exp_outcome/outcomes_12.12_%s_4"%label, gpu_device_num=0)
te = TextExplainer(random_state=42)
te.fit(text, predict_proba)
te.show_prediction(targets=[1])

Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done
Used for model gpu 0


In [159]:
from outcomes_modelling import outcomes_sentence_labeler_plus_NER
outcomes_sentence_labeler_plus_NER = reload(outcomes_sentence_labeler_plus_NER)
from outcomes_modelling import outcomes_full_logic_labeller
outcomes_full_logic_labeller = reload(outcomes_full_logic_labeller)
_outcomes_full_logic_labeller = outcomes_full_logic_labeller.OutcomesFullLogicLabeller(
    "/hdd2/data/maryia_pavlovets/bert_exp_outcome_new",
    "/hdd2/data/maryia_pavlovets/entity-recognition-model-more-2610",
    threshold=0.5)

In [10]:
df_to_eval = pd.DataFrame(outcomes_sentences_eval_data, columns=["text", "Label"])

In [160]:
df_to_eval["predicted"] = _outcomes_full_logic_labeller.predict_all_labels(df_to_eval["text"].values)[0]

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0224 10:57:40.081851 140362403956544 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Livelihood_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa330251a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0224 10:57:40.636495 140362403956544 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Livelihood_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa330251a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 291


...

I0224 10:59:26.718687 140362403956544 saver.py:1270] Restoring parameters from /hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Environment impact_3_1/model.ckpt-97


INFO:tensorflow:Running local_init_op.


I0224 10:59:27.460046 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 10:59:27.550498 140362403956544 session_manager.py:493] Done running local_init_op.


In [161]:
labels = ['Livelihood', 'Soil health', 'Fertilizer use', 'Water use', 'Gender',
           'Greenhouse gas emissions', 'Market access', 'Nutrition',
           'Production', 'Resilience', 'Knowledge sharing', 'Practice change',
           'Social inclusion', 'Poverty reduction', 'Environment impact']
labels2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}
#test_res = []
res_prob = []
for i in range(len(df_to_eval)):
    real_labels = set()
    for _label in df_to_eval["Label"].values[i]:
        real_labels.add(labels2id[_label]+1)
    if not real_labels:
        real_labels.add(16)
    #test_res.append(real_labels)
    predicted_prob = []
    for _label in labels:
        if _label in df_to_eval["predicted"].values[i]:
            predicted_prob.append(1.0)
        else:
            predicted_prob.append(0.0)
    res_prob.append(predicted_prob)

In [162]:
print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))

F1 measure:  0.7308296514482083
Accuracy:  0.7938144329896907


In [5]:
from outcomes_modelling import outcomes_multi_label_predictor
outcomes_multi_label_predictor = reload(outcomes_multi_label_predictor)
_outcomes_multi_label_predictor = outcomes_multi_label_predictor.OutcomesMultiLabelPredictor(
    "/hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7")

Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done


In [ ]:
from time import time
for folder in ["processed_usaid_pdfs"]: #["subset_usaid", "dfs_data","teams_dataset","big_dataset_1"]:
    for filename in os.listdir(os.path.join("../tmp", folder)):
        print(folder, filename)
        new_folder = (folder+"_new") if folder in ["big_dataset_1", "processed_usaid_pdfs"] else folder
        print(new_folder)
        if folder != new_folder and os.path.exists(os.path.join("../tmp", new_folder, filename)):
            print("Already processed")
            continue
        start = time()
        df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp", folder, filename))
        print(len(df))
        df["text"] = df["sentence"] #df["title"] + " . " + df["abstract"]
        found_labels, outcome_details = _outcomes_multi_label_predictor.predict_all_labels(df["text"].values)
        df["outcomes_found"] = found_labels
        df["outcomes_details"] = ["\n".join(detail) for detail in outcome_details]
        df = df.drop(["text"], axis=1)
        print("Finished ", (time() - start))
        os.makedirs(os.path.join("../tmp", new_folder), exist_ok=True)
        excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join("../tmp", new_folder, filename))

In [2]:
labels = ['Livelihood', 'Soil health', 'Fertilizer use', 'Water use', 'Gender',
           'Greenhouse gas emissions', 'Market access', 'Nutrition',
           'Production', 'Resilience', 'Knowledge sharing', 'Practice change',
           'Social inclusion', 'Poverty reduction', 'Environment impact']
labels2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}
def prepare_multilabel_data(outcomes_train_label_data):
    gathred_data = []
    dict_label = {}
    for i in range(len(outcomes_train_label_data)):
        if outcomes_train_label_data[i][0] not in dict_label:
            dict_label[outcomes_train_label_data[i][0]] = set()
        dict_label[outcomes_train_label_data[i][0]].add(outcomes_train_label_data[i][1])
    for key, value in dict_label.items():
        gathred_data.append((key, [(1 if label in value else 0) for label in labels]))
    return pd.DataFrame(gathred_data, columns=["text", "label"])

train_df = prepare_multilabel_data(outcomes_train_label_data)
test_df = prepare_multilabel_data(outcomes_eval_label_data)

In [24]:
all_sentences = excel_reader.ExcelReader().read_df_from_excel("../tmp/all_sentences.xlsx")

Read file ../tmp/all_sentences.xlsx: 25.94s
Processed file ../tmp/all_sentences.xlsx: 36.46s


In [27]:
all_sentences_part = shuffle(all_sentences[all_sentences["Taken"]==0])[:50000]

In [30]:
all_sentences_part = pd.DataFrame(all_sentences_part["abstract"].values, columns=["text"])

In [123]:
all_sentences_part["predicted"] = _outcomes_full_logic_labeller.predict_all_labels(all_sentences_part["text"].values)[0]

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0223 20:23:27.735594 140362403956544 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Livelihood_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa7d0158dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0223 20:23:28.177777 140362403956544 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Livelihood_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa7d0158dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 50020


...

I0223 21:59:42.523229 140362403956544 saver.py:1270] Restoring parameters from /hdd2/data/maryia_pavlovets/bert_exp_outcome_new/outcomes_12.12_Environment impact_3_1/model.ckpt-97


INFO:tensorflow:Running local_init_op.


I0223 21:59:43.484974 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0223 21:59:43.662018 140362403956544 session_manager.py:493] Done running local_init_op.


In [124]:
excel_writer.ExcelWriter().save_df_in_excel(all_sentences_part, "50000_docs_outcomes_labelled_0.7_thresh.xlsx")

Saving...
Saved to 50000_docs_outcomes_labelled_0.7_thresh.xlsx


In [125]:
dict_outcomes = {}
for i in range(len(all_sentences_part)):
    for label in all_sentences_part["predicted"].values[i]:
        if label not in dict_outcomes:
            dict_outcomes[label] = 0
        if len(all_sentences_part["predicted"].values[i]) == 1:
            dict_outcomes[label] += 1
dict_outcomes

{'Market access': 1201,
 'Production': 2699,
 'Resilience': 623,
 'Gender': 336,
 'Livelihood': 2261,
 'Water use': 253,
 'Nutrition': 760,
 'Fertilizer use': 665,
 'Environment impact': 916,
 'Greenhouse gas emissions': 396,
 'Soil health': 499,
 'Poverty reduction': 345,
 'Practice change': 305,
 'Knowledge sharing': 331,
 'Social inclusion': 103}

In [126]:
ind_to_take = []
dict_outcomes_to_take = {}
for i in range(len(all_sentences_part)):
    for label in all_sentences_part["predicted"].values[i]:
        if label not in ['Soil health',
           'Greenhouse gas emissions', 'Resilience', 'Knowledge sharing', 'Practice change', 'Environment impact',
            'Water use', 'Social inclusion', 'Gender', 'Fertilizer use', 'Poverty reduction', 'Nutrition', 'Market access']:
            continue
        if label not in dict_outcomes_to_take:
            dict_outcomes_to_take[label] = 0
        if len(all_sentences_part["predicted"].values[i]) == 1 and dict_outcomes_to_take[label] < 100:
            dict_outcomes_to_take[label] += 1
            ind_to_take.append(i)
print(len(ind_to_take), dict_outcomes_to_take)
df_part_to_add = []
for i in ind_to_take:
    df_part_to_add.append((all_sentences_part["text"].values[i], all_sentences_part["predicted"].values[i]))

1300 {'Market access': 100, 'Resilience': 100, 'Gender': 100, 'Water use': 100, 'Nutrition': 100, 'Fertilizer use': 100, 'Environment impact': 100, 'Greenhouse gas emissions': 100, 'Soil health': 100, 'Poverty reduction': 100, 'Practice change': 100, 'Knowledge sharing': 100, 'Social inclusion': 100}


In [127]:
ind_to_take = []
dict_outcomes_to_take = {}
for i in range(len(all_sentences_part)):
    for label in all_sentences_part["predicted"].values[i]:
        if label not in ['Soil health',
           'Greenhouse gas emissions', 'Resilience', 'Knowledge sharing', 'Practice change', 'Environment impact',
            'Water use', 'Social inclusion', 'Gender', 'Fertilizer use', 'Poverty reduction', 'Nutrition', 'Market access']:
            continue
        if label not in dict_outcomes_to_take:
            dict_outcomes_to_take[label] = {"only": 0, "with others": 0}
        if len(all_sentences_part["predicted"].values[i]) == 1 and dict_outcomes_to_take[label]["only"] < 30:
            dict_outcomes_to_take[label]["only"] += 1
            ind_to_take.append(i)
        if len(all_sentences_part["predicted"].values[i]) > 1 and dict_outcomes_to_take[label]["with others"] < 70:
            dict_outcomes_to_take[label]["with others"] += 1
            if i not in ind_to_take:
                ind_to_take.append(i)
print(len(ind_to_take), dict_outcomes_to_take)
df_part_to_add = []
for i in ind_to_take:
    df_part_to_add.append((all_sentences_part["text"].values[i], all_sentences_part["predicted"].values[i]))

1068 {'Market access': {'only': 30, 'with others': 70}, 'Resilience': {'only': 30, 'with others': 70}, 'Gender': {'only': 30, 'with others': 70}, 'Water use': {'only': 30, 'with others': 70}, 'Nutrition': {'only': 30, 'with others': 70}, 'Fertilizer use': {'only': 30, 'with others': 70}, 'Environment impact': {'only': 30, 'with others': 70}, 'Greenhouse gas emissions': {'only': 30, 'with others': 70}, 'Soil health': {'only': 30, 'with others': 70}, 'Poverty reduction': {'only': 30, 'with others': 70}, 'Practice change': {'only': 30, 'with others': 70}, 'Knowledge sharing': {'only': 30, 'with others': 70}, 'Social inclusion': {'only': 30, 'with others': 63}}


In [18]:
from outcomes_modelling import outcomes_extracted_labeler
outcomes_extracted_labeler = reload(outcomes_extracted_labeler)

_outcomes_extracted_labeler = outcomes_extracted_labeler.OutcomesExtractedLabeler("/hdd2/data/maryia_pavlovets/bert_exp_outcome_new_multilabel_9epoch",
                                                                                  gpu_device_num=0, multilabel=True, label_list=list(range(len(labels))),
                                                                                 epoch_num=9)

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0214 18:40:16.676713 140597353092928 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new_multilabel_9epoch', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde846de358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0214 18:40:17.013569 140597353092928 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_new_multilabel_9epoch', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde846de358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [19]:
res = _outcomes_extracted_labeler.train_model(train_df, test_df)

INFO:tensorflow:Writing example 0 of 2488


I0214 18:40:17.058700 140597353092928 base_bert_model.py:478] Writing example 0 of 2488


INFO:tensorflow:*** Example ***


I0214 18:40:17.061357 140597353092928 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Saving checkpoints for 699 into /hdd2/data/maryia_pavlovets/bert_exp_outcome_new_multilabel_9epoch/model.ckpt.


I0214 18:44:52.474623 140597353092928 basic_session_run_hooks.py:594] Saving checkpoints for 699 into /hdd2/data/maryia_pavlovets/bert_exp_outcome_new_multilabel_9epoch/model.ckpt.


INFO:tensorflow:Loss for final step: 0.039357975.


I0214 18:44:54.175347 140597353092928 estimator.py:359] Loss for final step: 0.039357975.


Training took time  0:04:36.814615


In [8]:
from sklearn.metrics import f1_score
from text_processing import text_normalizer

def get_f1_score_test_data(test_data, intervention_labeler):
    res_pred, res_prob = intervention_labeler.predict_class(test_data.values, return_probs=True)
    res_true = test_data["Label"].values
    return f1_score(res_true, res_pred, average="macro")

def get_f1_multi_label_new(test_labels, y_true):
    final_res = []
    for i in range(len(test_labels)):
        identified_labels = set()
        for j in range(len(test_labels[i])):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(16)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (cnt_intersect/cnt_found)
        recall = 0 if cnt_correct == 0 else (cnt_intersect/cnt_correct)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def get_accuracy_multi_label_new(test_labels, y_true):
    final_res = []
    for i in range(len(test_labels)):
        identified_labels = set()
        for j in range(len(test_labels[i])):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(16)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (1 if cnt_intersect > 0 else 0)
        recall = 0 if cnt_correct == 0 else (1 if cnt_intersect > 0 else 0)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def normalize_full(text):
    return " ".join(sorted(text_normalizer.normalize_sentence(text).split(" ")))

def deduplicate(dfs):
    dicts = {}
    for df in dfs:
        for i in range(len(df)):
            processed = normalize_full(df["Narrow concept"].values[i])
            if processed not in dicts:
                dicts[processed] = set()
            dicts[processed].add(df["Label"].values[i])
    return dicts

def print_per_label_stats(test_res, res_label):
    from sklearn.metrics import f1_score
    f1_score_ = 0.0
    for idx, label in enumerate(labels):
        print(label)
        test_y = [1 if (idx+1) in test_res[i] else 0 for i in range(len(test_res))]
        res_y = [res_label[i][idx] for i in range(len(test_res))]
        _outcomes_sentence_labeler.print_summary(test_y, res_y)
        f1_score_ += f1_score(test_y, res_y, average="macro")
    print("Average ", f1_score_/15)

In [13]:
from outcomes_modelling import outcomes_sentence_labeler
outcomes_sentence_labeler = reload(outcomes_sentence_labeler)
_outcomes_sentence_labeler = outcomes_sentence_labeler.OutcomesSentenceLabeler("/hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7",
                                                                                  gpu_device_num=0, multilabel=True, label_list=list(range(len(labels))),
                                                                                 epoch_num=15)

Started tokenizer loading
Used gpu 0
Instructions for updating:
Colocations handled automatically by placer.


W0312 15:09:35.860630 140421369902912 deprecation.py:323] From /home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0312 15:09:37.114621 140421369902912 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb58afbdeb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0312 15:09:37.728572 140421369902912 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb58afbdeb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [1]:
labels = ['Livelihood', 'Soil health', 'Fertilizer use', 'Water use', 'Gender',
           'Greenhouse gas emissions', 'Market access', 'Nutrition',
           'Production', 'Resilience', 'Knowledge sharing', 'Practice change',
           'Social inclusion', 'Poverty reduction', 'Environment impact']
labels2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}
def prepare_multilabel_data_sentences(outcomes_train_label_data):
    gathred_data = []
    for i in range(len(outcomes_train_label_data)):
        gathred_data.append((outcomes_train_label_data[i][0], [(1 if label in outcomes_train_label_data[i][1] else 0) for label in labels]))
    return pd.DataFrame(gathred_data, columns=["text", "label"])

train_df = prepare_multilabel_data_sentences(outcomes_sentences_train_data + df_part_to_add)
test_df = prepare_multilabel_data_sentences(outcomes_sentences_eval_data)
test_res = []
for i in range(len(test_df)):
    id_labels = set()
    for _idx in range(len(test_df["label"].values[i])):
        if test_df["label"].values[i][_idx] == 1:
            id_labels.add(_idx + 1)
    if not id_labels:
        id_labels.add(16)
    test_res.append(id_labels)

In [130]:
train_df = shuffle(train_df)

In [131]:
outcomes_sentences_dict = {}
part_data = outcomes_sentences_train_data + df_part_to_add
for i in range(len(part_data)):
    for label in set(part_data[i][1]):
        if label not in outcomes_sentences_dict:
            outcomes_sentences_dict[label] = 0
        outcomes_sentences_dict[label] += 1
sorted(outcomes_sentences_dict.items(), key=lambda x:x[1], reverse=True)

[('Livelihood', 593),
 ('Production', 577),
 ('Nutrition', 311),
 ('Resilience', 294),
 ('Greenhouse gas emissions', 289),
 ('Market access', 283),
 ('Soil health', 277),
 ('Environment impact', 250),
 ('Practice change', 246),
 ('Fertilizer use', 223),
 ('Knowledge sharing', 222),
 ('Gender', 220),
 ('Water use', 217),
 ('Poverty reduction', 216),
 ('Social inclusion', 196)]

In [132]:
res = _outcomes_sentence_labeler.train_model(train_df, test_df)

INFO:tensorflow:Writing example 0 of 3678


I0223 22:07:02.658501 140362403956544 base_bert_model.py:478] Writing example 0 of 3678


INFO:tensorflow:*** Example ***


I0223 22:07:02.660722 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Saving checkpoints for 1724 into /hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7/model.ckpt.


I0223 22:30:49.877268 140362403956544 basic_session_run_hooks.py:594] Saving checkpoints for 1724 into /hdd2/data/maryia_pavlovets/bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7/model.ckpt.


INFO:tensorflow:Loss for final step: 0.026863242.


I0223 22:30:51.573641 140362403956544 estimator.py:359] Loss for final step: 0.026863242.


Training took time  0:23:47.427477


In [177]:
#bert_exp_outcome_sentences_new_multilabel_15epoch_1300_mixed_0.7
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:19:08.335257 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:19:08.337956 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:19:11.699921 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:19:11.794755 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7356106474663178
Accuracy:  0.7972508591065293
Livelihood
[[246   7]
 [  8  30]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       253
           1       0.81      0.79      0.80        38

   micro avg       0.95      0.95      0.95       291
   macro avg       0.89      0.88      0.89       291
weighted avg       0.95      0.95      0.95       291

F1 score:  0.8852071005917159
Soil health
[[271   3]
 [  8   9]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       274
           1       0.75      0.53      0.62        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.86      0.76      0.80       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.8003990771341274
Fertilizer use
[[273   5]
 [  3  10]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       278
        

In [179]:
#bert_exp_outcome_sentences_new_multilabel_10epoch_1300_mixed
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:20:26.760685 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:20:26.762917 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:20:30.113404 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:20:30.206860 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7339142529864182
Accuracy:  0.7938144329896907
Livelihood
[[248   5]
 [  8  30]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       253
           1       0.86      0.79      0.82        38

   micro avg       0.96      0.96      0.96       291
   macro avg       0.91      0.88      0.90       291
weighted avg       0.95      0.96      0.95       291

F1 score:  0.898188766585031
Soil health
[[271   3]
 [  8   9]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       274
           1       0.75      0.53      0.62        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.86      0.76      0.80       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.8003990771341274
Fertilizer use
[[274   4]
 [  5   8]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       278
         

In [181]:
#bert_exp_outcome_sentences_new_multilabel_12epoch_1300
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:21:14.589338 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:21:14.593148 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:21:17.951717 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:21:18.049754 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7233213331151478
Accuracy:  0.7835051546391752
Livelihood
[[247   6]
 [ 11  27]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       253
           1       0.82      0.71      0.76        38

   micro avg       0.94      0.94      0.94       291
   macro avg       0.89      0.84      0.86       291
weighted avg       0.94      0.94      0.94       291

F1 score:  0.8636476392602188
Soil health
[[268   6]
 [  9   8]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       274
           1       0.57      0.47      0.52        17

   micro avg       0.95      0.95      0.95       291
   macro avg       0.77      0.72      0.74       291
weighted avg       0.94      0.95      0.95       291

F1 score:  0.7444529008840233
Fertilizer use
[[276   2]
 [  5   8]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       278
        

In [183]:
#bert_exp_outcome_sentences_new_multilabel_15epoch_1000
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:21:50.470282 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:21:50.474194 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:21:53.930803 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:21:54.037990 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7141657121038566
Accuracy:  0.7766323024054983
Livelihood
[[247   6]
 [ 10  28]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       253
           1       0.82      0.74      0.78        38

   micro avg       0.95      0.95      0.95       291
   macro avg       0.89      0.86      0.87       291
weighted avg       0.94      0.95      0.94       291

F1 score:  0.873202614379085
Soil health
[[269   5]
 [ 10   7]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       274
           1       0.58      0.41      0.48        17

   micro avg       0.95      0.95      0.95       291
   macro avg       0.77      0.70      0.73       291
weighted avg       0.94      0.95      0.94       291

F1 score:  0.727816923364719
Fertilizer use
[[274   4]
 [  6   7]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       278
          

In [185]:
#bert_exp_outcome_sentences_new_multilabel_15epoch_new_data_600
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:22:27.414615 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:22:27.417113 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:22:30.844191 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:22:30.951570 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7415425735013366
Accuracy:  0.8041237113402062
Livelihood
[[247   6]
 [  7  31]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       253
           1       0.84      0.82      0.83        38

   micro avg       0.96      0.96      0.96       291
   macro avg       0.91      0.90      0.90       291
weighted avg       0.95      0.96      0.96       291

F1 score:  0.9005128205128204
Soil health
[[270   4]
 [  9   8]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       274
           1       0.67      0.47      0.55        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.82      0.73      0.76       291
weighted avg       0.95      0.96      0.95       291

F1 score:  0.7641080002494232
Fertilizer use
[[276   2]
 [  6   7]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       278
        

In [187]:
# 20 epoch bert_exp_outcome_sentences_new_multilabel_20epoch
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:23:24.397002 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:23:24.400151 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:23:27.926547 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:23:28.020484 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7254540991654396
Accuracy:  0.7835051546391752
Livelihood
[[248   5]
 [ 10  28]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       253
           1       0.85      0.74      0.79        38

   micro avg       0.95      0.95      0.95       291
   macro avg       0.90      0.86      0.88       291
weighted avg       0.95      0.95      0.95       291

F1 score:  0.8796890934648991
Soil health
[[272   2]
 [  9   8]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       274
           1       0.80      0.47      0.59        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.88      0.73      0.79       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.7863863863863865
Fertilizer use
[[277   1]
 [  6   7]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       278
        

In [189]:
# 15 epoch bert_exp_outcome_sentences_new_multilabel_15epoch
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0224 11:23:59.676289 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0224 11:23:59.678941 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0224 11:24:03.080127 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0224 11:24:03.184670 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.7223367697594504
Accuracy:  0.7766323024054983
Livelihood
[[249   4]
 [  9  29]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       253
           1       0.88      0.76      0.82        38

   micro avg       0.96      0.96      0.96       291
   macro avg       0.92      0.87      0.90       291
weighted avg       0.95      0.96      0.95       291

F1 score:  0.8957305476695792
Soil health
[[272   2]
 [  9   8]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       274
           1       0.80      0.47      0.59        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.88      0.73      0.79       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.7863863863863865
Fertilizer use
[[276   2]
 [  6   7]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       278
        

In [74]:
# 12 epoch bert_exp_outcome_sentences_new_multilabel_12epoch
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0223 17:02:42.687530 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0223 17:02:42.691551 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0223 17:02:48.387642 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0223 17:02:48.470529 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.6930889652539138
Accuracy:  0.7525773195876289
Livelihood
[[242   7]
 [ 12  30]]
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       249
           1       0.81      0.71      0.76        42

   micro avg       0.93      0.93      0.93       291
   macro avg       0.88      0.84      0.86       291
weighted avg       0.93      0.93      0.93       291

F1 score:  0.8608601555225608
Soil health
[[273   1]
 [ 10   7]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       274
           1       0.88      0.41      0.56        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.92      0.70      0.77       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.7701256732495512
Fertilizer use
[[277   1]
 [  7   6]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       278
        

In [76]:
#9 epoch bert_exp_outcome_sentences_new_multilabel_9epoch
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0223 17:02:59.242354 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0223 17:02:59.245996 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0223 17:03:04.970377 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0223 17:03:05.052742 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.6863688430698742
Accuracy:  0.7491408934707904
Livelihood
[[244   5]
 [ 15  27]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       249
           1       0.84      0.64      0.73        42

   micro avg       0.93      0.93      0.93       291
   macro avg       0.89      0.81      0.85       291
weighted avg       0.93      0.93      0.93       291

F1 score:  0.8451798254947862
Soil health
[[273   1]
 [ 10   7]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       274
           1       0.88      0.41      0.56        17

   micro avg       0.96      0.96      0.96       291
   macro avg       0.92      0.70      0.77       291
weighted avg       0.96      0.96      0.96       291

F1 score:  0.7701256732495512
Fertilizer use
[[276   2]
 [  6   7]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       278
        

In [78]:
# 3 epoch bert_exp_outcome_sentences_new_multilabel_3epoch
res_prob, res_label, _ = _outcomes_sentence_labeler.predict_for_df(test_df)

print("F1 measure: ", get_f1_multi_label_new(res_prob, test_res))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, test_res))
print_per_label_stats(test_res, res_label)

INFO:tensorflow:Writing example 0 of 291


I0223 17:03:24.354169 140362403956544 base_bert_model.py:478] Writing example 0 of 291


INFO:tensorflow:*** Example ***


I0223 17:03:24.357713 140362403956544 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

I0223 17:03:30.037300 140362403956544 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0223 17:03:30.119460 140362403956544 session_manager.py:493] Done running local_init_op.


F1 measure:  0.513860252004582
Accuracy:  0.5463917525773195
Livelihood
[[244   5]
 [ 30  12]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.93       249
           1       0.71      0.29      0.41        42

   micro avg       0.88      0.88      0.88       291
   macro avg       0.80      0.63      0.67       291
weighted avg       0.86      0.88      0.86       291

F1 score:  0.6699290274492011
Soil health
[[273   1]
 [ 14   3]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       274
           1       0.75      0.18      0.29        17

   micro avg       0.95      0.95      0.95       291
   macro avg       0.85      0.59      0.63       291
weighted avg       0.94      0.95      0.93       291

F1 score:  0.6294881588999236
Fertilizer use
[[278   0]
 [ 13   0]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       278
         

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Other experiments with outcomes finding

In [328]:
from outcomes_modelling import outcomes_sentence_labeler
outcomes_sentence_labeler = reload(outcomes_sentence_labeler)
_outcomes_sentence_labeler = outcomes_sentence_labeler.OutcomesSentenceLabeler("outcomes_new_%s_4"%label_to_train, gpu_device_num=0)

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1106 14:56:33.299341 139697590273856 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': 'outcomes_new_Livelihood_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c2c2de400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I1106 14:56:33.661720 139697590273856 estimator.py:201] Using config: {'_model_dir': 'outcomes_new_Livelihood_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0c2c2de400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done


In [329]:
train_data_df = _outcomes_sentence_labeler.prepare_df_data_sentences(outcomes_sentences_train_data, label_to_train, include_empty_sentences=True)
test_data_df = _outcomes_sentence_labeler.prepare_df_data_sentences(outcomes_sentences_eval_data, label_to_train, include_empty_sentences=True)

In [330]:
res = _outcomes_sentence_labeler.train_model(train_data_df, test_data_df)

INFO:tensorflow:Writing example 0 of 1674


I1106 14:56:49.445896 139697590273856 base_bert_model.py:424] Writing example 0 of 1674


INFO:tensorflow:*** Example ***


I1106 14:56:49.447578 139697590273856 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I1106 15:03:26.891546 139697590273856 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I1106 15:03:26.961287 139697590273856 session_manager.py:493] Done running local_init_op.


[[128   3]
 [  3  17]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       131
           1       0.85      0.85      0.85        20

   micro avg       0.96      0.96      0.96       151
   macro avg       0.91      0.91      0.91       151
weighted avg       0.96      0.96      0.96       151

F1 score:  0.9135496183206107


In [358]:
from outcomes_modelling import outcomes_sentence_labeler_plus_NER
outcomes_sentence_labeler_plus_NER = reload(outcomes_sentence_labeler_plus_NER)

_outcomes_sentence_labeler_plus_NER = outcomes_sentence_labeler_plus_NER.OutcomesSentenceLabelerPlusNer(
    "outcomes_new_%s_1"%label_to_train, path_to_NER="entity-recognition-model-more-1664", gpu_device_num=0)

Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done


In [342]:
train_data_df = _outcomes_sentence_labeler_plus_NER.prepare_df_data_sentences(outcomes_sentences_train_data, label_to_train, include_empty_sentences=True)
test_data_df = _outcomes_sentence_labeler_plus_NER.prepare_df_data_sentences(outcomes_sentences_eval_data, label_to_train, include_empty_sentences=True)

In [359]:
res = _outcomes_sentence_labeler_plus_NER.evaluate_model(test_data_df)

Used for model gpu 1
[[130   1]
 [  8  12]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97       131
           1       0.92      0.60      0.73        20

   micro avg       0.94      0.94      0.94       151
   macro avg       0.93      0.80      0.85       151
weighted avg       0.94      0.94      0.93       151

F1 score:  0.8469077391010478


In [208]:
pred_sentences = [
    ("improve livelihood", ),
    ("maximize farmer welfare",),
    ("maximize welfare",),
    ("reduce income",),
    ("reduce GHG",),
    ("increase productivity",)
]

In [317]:
predictions = _outcomes_extracted_labeler.predict_for_df(pd.DataFrame(pred_sentences, columns=["text"]))

INFO:tensorflow:Writing example 0 of 6


I1106 14:38:42.553262 139697590273856 base_bert_model.py:424] Writing example 0 of 6


INFO:tensorflow:*** Example ***


I1106 14:38:42.556041 139697590273856 base_bert_model.py:400] *** Example ***


INFO:tensorflow:guid: 


...

I1106 14:38:45.277765 139697590273856 saver.py:1270] Restoring parameters from outcomes_new_Livelihood_1/model.ckpt-94


INFO:tensorflow:Running local_init_op.


I1106 14:38:45.848966 139697590273856 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I1106 14:38:45.915914 139697590273856 session_manager.py:493] Done running local_init_op.


In [319]:
for i in range(len(pred_sentences)):
    print(pred_sentences[i], predictions[0][i], predictions[1][i])

('improve livelihood',) [0.0280929 0.9719071] 1
('maximize farmer welfare',) [0.03571642 0.96428365] 1
('maximize welfare',) [0.0344816  0.96551836] 1
('reduce income',) [0.02902093 0.9709791 ] 1
('reduce GHG',) [0.9946977  0.00530234] 0
('increase productivity',) [0.9952474  0.00475256] 0
